In [2]:
from clip_image_to_text import load_interrogator, inference
model = load_interrogator("/home/mlfavorfit/lib/favorfit/kjg/0_model_weights/image_to_text/clip", caption_model_name="blip-base", device="cuda") 

In [4]:
from PIL import Image

img_pil = Image.open("/media/mlfavorfit/sdb/contolnet_dataset/val/158_Chane.jpg").convert('RGB')
result = inference(img_pil=img_pil, model=model)
print(result)

chan chan perfume bottle on a black background, perfume bottle, dramatic product photography, antique perfume, chaumet, floating in perfume, commercial product photography, chanel, dramatic product shot, miniature photography, product photography, rendered with unreal engine, product photography 4 k, 3 d product render, still life photography, commercial photo, packshot, behance. polished


In [5]:
from glob import glob

fns = glob("/media/mlfavorfit/sdb/contolnet_dataset/control_net_train_base/images/*")

In [6]:
result_dict_list = []

In [9]:
import os
from tqdm import tqdm

for fn in tqdm(fns, total=len(fns)):
    temp_dict = {"text":None, "image":None, "mask":None}

    img = Image.open(fn).convert("RGB")
    caption = inference(img_pil=img, model=model)

    temp_dict["text"] = caption
    temp_dict["image"] = os.path.join("images", os.path.basename(fn))
    temp_dict["mask"] = os.path.join("masks", os.path.basename(fn))

    result_dict_list.append(temp_dict)

100%|██████████| 197967/197967 [9:49:27<00:00,  5.60it/s]  


In [10]:
import json
def save_data_to_jsonlines(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            json.dump(item, f)
            f.write('\n')

In [11]:
save_data_to_jsonlines(result_dict_list, './temp.jsonl')